In [11]:
# Import warnings filter
from warnings import simplefilter

# Ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#Import packages

import copy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import where
from pprint import pprint
from scipy import stats
from collections import Counter

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV, \
RandomizedSearchCV, StratifiedKFold, RepeatedStratifiedKFold, cross_val_score

from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import make_scorer, recall_score, confusion_matrix, \
precision_score, f1_score, accuracy_score, classification_report


In [13]:
import psycopg2 as pg
import pandas.io.sql as psql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine
from sqlalchemy import inspect
import config 

## Read Data From PostgreSQL DB

In [14]:
# Conncet to postgresql dataBase table
protocol = 'postgresql'
username = 'postgres'
password = config.password
host = 'localhost'
port = 5432
database_name = 'Project4_db'

pg_connection = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'

In [15]:
#Create DataFrame from SQL table 
pg_connection = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
conn = pg.connect(pg_connection)
raw_ibm_df = psql.read_sql('SELECT * FROM ibm_employee_data', conn)

# Display all the DataFrame Columns 
pd.options.display.max_columns = None
display(raw_ibm_df)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,3,Male,41,4,2,Laboratory Technician,4,Married,2571,12290,4,Y,No,17,3,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,4,Male,42,2,3,Healthcare Representative,1,Married,9991,21457,4,Y,No,15,3,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,2,Male,87,4,2,Manufacturing Director,2,Married,6142,5174,1,Y,Yes,20,4,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,4,Male,63,2,2,Sales Executive,2,Married,5390,13243,2,Y,No,14,3,4,80,0,17,3,2,9,6,0,8


In [25]:
# Remane columns
raw_data = raw_ibm_df.rename(columns = {'age' : 'Age', 
                                        'attrition' :'Attrition', 
                                        'businesstravel' : 'BusinessTravel', 
                                        'dailyrate' : 'DailyRate', 
                                        'department' : 'Department',
                                        'distancefromhome' : 'DistanceFromHome', 
                                        'education' : 'Education', 
                                        'educationfield' : 'EducationField', 
                                        'employeecount' : 'EmployeeCount',
                                        'employeenumber' : 'EmployeeNumber', 
                                        'environmentsatisfaction' : 'EnvironmentSatisfaction', 
                                        'gender' : 'Gender', 
                                        'hourlyrate' : 'HourlyRate',
                                        'jobinvolvement' : 'JobInvolvement', 
                                        'joblevel' : 'JobLevel', 
                                        'jobrole' : 'JobRole', 
                                        'jobsatisfaction' : 'JobSatisfaction',
                                        'maritalstatus' : 'MaritalStatus', 
                                        'monthlyincome' : 'MonthlyIncome', 
                                        'monthlyrate' : 'MonthlyRate', 
                                        'numcompaniesworked' : 'NumCompaniesWorked',
                                        'over18' : 'Over18', 
                                        'overtime' : 'OverTime', 
                                        'percentsalaryhike' : 'PercentSalaryHike', 
                                        'performancerating' : 'PerformanceRating',
                                        'relationshipsatisfaction' : 'RelationshipSatisfaction', 
                                        'standardhours' : 'StandardHours', 
                                        'stockoptionlevel' : 'StockOptionLevel',
                                        'totalworkingyears' : 'TotalWorkingYears', 
                                        'trainingtimeslastyear' : 'TrainingTimesLastYear', 
                                        'worklifebalance' : 'WorkLifeBalance',
                                        'yearsatcompany' : 'YearsAtCompany', 
                                        'yearsincurrentrole': 'YearsInCurrentRole', 
                                        'yearssincelastpromotion' : 'YearsSinceLastPromotion',
                                        'yearswithcurrmanager' : 'YearsWithCurrManager'
})
display(raw_data)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,3,Male,41,4,2,Laboratory Technician,4,Married,2571,12290,4,Y,No,17,3,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,4,Male,42,2,3,Healthcare Representative,1,Married,9991,21457,4,Y,No,15,3,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,2,Male,87,4,2,Manufacturing Director,2,Married,6142,5174,1,Y,Yes,20,4,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,4,Male,63,2,2,Sales Executive,2,Married,5390,13243,2,Y,No,14,3,4,80,0,17,3,2,9,6,0,8


Data pre-processing

In [26]:
display(raw_data.head())
print()
print('There are', raw_data.shape[0], 'observations and', raw_data.shape[1], 'features')

# Attrition column: stay is 0, churn is 1 
# Gender column: female is 0, male is 1


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2



There are 1470 observations and 35 features


In [27]:
raw_data.dtypes

Age                          int64
Attrition                   object
BusinessTravel              object
DailyRate                    int64
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeNumber              object
EnvironmentSatisfaction      int64
Gender                      object
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
MonthlyIncome                int64
MonthlyRate                  int64
NumCompaniesWorked           int64
Over18                      object
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears   

One-hot encoding

In [28]:
# Encoding categorical variables
data = pd.get_dummies(raw_data, columns=None)
data.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,EmployeeNumber_1,EmployeeNumber_10,EmployeeNumber_100,EmployeeNumber_1001,EmployeeNumber_1002,EmployeeNumber_1003,EmployeeNumber_1004,EmployeeNumber_1005,EmployeeNumber_1006,EmployeeNumber_1007,EmployeeNumber_1009,EmployeeNumber_101,EmployeeNumber_1010,EmployeeNumber_1011,EmployeeNumber_1012,EmployeeNumber_1013,EmployeeNumber_1014,EmployeeNumber_1015,EmployeeNumber_1016,EmployeeNumber_1017,EmployeeNumber_1018,EmployeeNumber_1019,EmployeeNumber_102,EmployeeNumber_1022,EmployeeNumber_1024,EmployeeNumber_1025,EmployeeNumber_1026,EmployeeNumber_1027,EmployeeNumber_1028,EmployeeNumber_1029,EmployeeNumber_103,EmployeeNumber_1030,EmployeeNumber_1032,EmployeeNumber_1033,EmployeeNumber_1034,EmployeeNumber_1035,EmployeeNumber_1036,EmployeeNumber_1037,EmployeeNumber_1038,EmployeeNumber_1039,EmployeeNumber_104,EmployeeNumber_1040,EmployeeNumber_1042,EmployeeNumber_1043,EmployeeNumber_1044,EmployeeNumber_1045,EmployeeNumber_1046,EmployeeNumber_1047,EmployeeNumber_1048,EmployeeNumber_1049,EmployeeNumber_105,EmployeeNumber_1050,EmployeeNumber_1052,EmployeeNumber_1053,EmployeeNumber_1055,EmployeeNumber_1056,EmployeeNumber_106,EmployeeNumber_1060,EmployeeNumber_1061,EmployeeNumber_1062,EmployeeNumber_1066,EmployeeNumber_1068,EmployeeNumber_1069,EmployeeNumber_107,EmployeeNumber_1070,EmployeeNumber_1071,EmployeeNumber_1073,EmployeeNumber_1074,EmployeeNumber_1076,EmployeeNumber_1077,EmployeeNumber_1079,EmployeeNumber_1080,EmployeeNumber_1081,EmployeeNumber_1082,EmployeeNumber_1083,EmployeeNumber_1084,EmployeeNumber_1085,EmployeeNumber_1088,EmployeeNumber_1092,EmployeeNumber_1094,EmployeeNumber_1096,EmployeeNumber_1097,EmployeeNumber_1098,EmployeeNumber_1099,EmployeeNumber_11,EmployeeNumber_110,EmployeeNumber_1100,EmployeeNumber_1101,EmployeeNumber_1102,EmployeeNumber_1103,EmployeeNumber_1105,EmployeeNumber_1106,EmployeeNumber_1107,EmployeeNumber_1108,EmployeeNumber_1109,EmployeeNumber_1111,EmployeeNumber_1113,EmployeeNumber_1114,EmployeeNumber_1115,EmployeeNumber_1116,EmployeeNumber_1117,EmployeeNumber_1118,EmployeeNumber_1119,EmployeeNumber_112,EmployeeNumber_1120,EmployeeNumber_1121,EmployeeNumber_1124,EmployeeNumber_1125,EmployeeNumber_1126,EmployeeNumber_1127,EmployeeNumber_1128,EmployeeNumber_113,EmployeeNumber_1131,EmployeeNumber_1132,EmployeeNumber_1133,EmployeeNumber_1135,EmployeeNumber_1136,EmployeeNumber_1137,EmployeeNumber_1138,EmployeeNumber_1140,EmployeeNumber_1143,EmployeeNumber_1148,EmployeeNumber_1150,EmployeeNumber_1152,EmployeeNumber_1154,EmployeeNumber_1156,EmployeeNumber_1157,EmployeeNumber_1158,EmployeeNumber_116,EmployeeNumber_1160,EmployeeNumber_1161,EmployeeNumber_1162,EmployeeNumber_1163,EmployeeNumber_1164,EmployeeNumber_1165,EmployeeNumber_1166,EmployeeNumber_1167,EmployeeNumber_117,EmployeeNumber_1171,EmployeeNumber_1172,EmployeeNumber_1173,EmployeeNumber_1175,EmployeeNumber_1177,EmployeeNumber_1179,EmployeeNumber_118,EmployeeNumber_1180,EmployeeNumber_1182,EmployeeNumber_1184,EmployeeNumber_1185,EmployeeNumber_1188,EmployeeNumber_119,EmployeeNumber_1190,EmployeeNumber_1191,EmployeeNumber_1192,EmployeeNumber_1193,EmployeeNumber_1195,EmployeeNumber_1196,EmployeeNumber_1198,EmployeeNumber_12,EmployeeNumber_120,EmployeeNumber_1200,EmployeeNumber_1201,EmployeeNumber_1202,Employee

In [23]:
data.columns

Index(['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction',
       ...
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'JobRole_Sales Representative',
       'MaritalStatus_Divorced', 'MaritalStatus_Married',
       'MaritalStatus_Single', 'Over18_Y', 'OverTime_No', 'OverTime_Yes'],
      dtype='object', length=1526)

Standardizing values

In [ ]:
scaler = MinMaxScaler()
df = scaler.fit_transform(data.values)
scaled = pd.DataFrame(df, columns=data.columns)

# There are now 47 features, since we broke down categorical vars
scaled.tail()

Checking data imbalance

In [ ]:
sns.countplot(x=data['Attrition'])
plt.title('Data count by target variable Y (Attrition)')
plt.show()